In [1]:
# ------ 요청사항 ------

# 0. 파일 경로 
# file_path = r"D:\pythondata\★_재배치_회수.csv"
# file_path = r"D:\pythondata\★_재배치_배송.csv"
# 1. 먼저, 두 파일을 하나의 파일로 합친 후 
# 2. 회수 파일과 배송 파일을 구분하기 위해서 "구분"이라는 이름의 필드를 마지막 컬럼으로 생성해서 적어두면 좋겠어요.
# 3. 원본데이터 필드는 "자전거번호,대여소,대여소명,날짜,시간"이고, "날짜+시간"이라는 필드를 만들어 주세요.
# 4. "대여소, 날짜, 시간"으로 정렬(sorting)하고 시작하는데,
# 5. No라는 컬럼을 가장 왼쪽에 생성하고, 정렬 순서(단순 정렬 개념)에 따라서 제일 윗줄부터 1로 시작하면 좋겠어요.
# 6. 이 파일을 "5-1_회수배송_데이터(1)"라는 이름으로 csv파일을 저장해 주세요.
# 7. 정리하면, "두 개 파일(★_재배치_회수.csv, ★_재배치_배송.csv)을 합쳐서 "구분"이라는 이름의 필드와 "No"라는 이름의 필드, "날짜+시간" 필드를 새로 만든다. 
#    "대여소"가 낮은 순서(102번부터 시작)이면서 "날짜+시간" 필드가 빠른 순서(오름차순 정렬, 2023년 1월 1일 0시 0분 0초부터)로 정렬. 정렬된 순서에 따라서 "No"의 번호 넣기" 입니다!

In [2]:
import pandas as pd

# 0. 파일 경로
path_retrieve = r"D:\pythondata\★_재배치_회수.csv"
path_delivery = r"D:\pythondata\★_재배치_배송.csv"

# 1. 파일 불러오기 (탭 구분자 + 인코딩 처리 + 대여소 문자 처리)
df_retrieve = pd.read_csv(path_retrieve, encoding='utf-8-sig', sep="\t", dtype={'대여소': str})
df_delivery = pd.read_csv(path_delivery, encoding='utf-8-sig', sep="\t", dtype={'대여소': str})

# 2. '구분' 필드 추가 (일단 추가는 지금 하고, 나중에 마지막 컬럼으로 이동)
df_retrieve['구분'] = '회수'
df_delivery['구분'] = '배송'

# 3. 병합
df = pd.concat([df_retrieve, df_delivery], ignore_index=True)

# 4. '날짜+시간' 필드 생성
df['날짜+시간'] = pd.to_datetime(df['날짜'] + ' ' + df['시간'])

# 5. 정렬 기준: ① 대여소 번호를 숫자로 변환해서 정렬 (102가 위로) ② 날짜+시간 오름차순
df['대여소_정렬용'] = pd.to_numeric(df['대여소'], errors='coerce')  # 숫자로 변환 불가능한 것은 NaN 처리
df = df.sort_values(by=['대여소_정렬용', '날짜+시간']).reset_index(drop=True)
df.drop(columns='대여소_정렬용', inplace=True)  # 임시 정렬 컬럼 제거

# 6. No 필드 추가 (가장 왼쪽에 삽입)
df.insert(0, 'No', df.index + 1)

# 7. 컬럼 순서 정리: '구분'은 마지막으로 이동
cols = [col for col in df.columns if col != '구분'] + ['구분']
df = df[cols]

# 8. 저장 및 완료 메시지
output_path = r"D:\pythondata\5-1_회수배송_데이터(1).csv"
df.to_csv(output_path, index=False, encoding='utf-8-sig')

print("✅ 5-1 파일 저장 완료!", output_path)

✅ 5-1 파일 저장 완료! D:\pythondata\5-1_회수배송_데이터(1).csv


In [3]:
# ------ 요청사항 ------

# 8. file_path = r"D:\pythondata\5-1_회수배송_데이터(1).csv"를 다시 생성하고,
# 9. 추가해서 만들 필드는 "방문코드,순번,flag,자전거대수,first data&time,last data&time" 6개예요.
# 10. 필드의 순서는 총 14개 "No,자전거번호,대여소,대여소명,날짜,시간,날짜+시간,방문코드,순번,flag,자전거대수,first data&time,last data&time,구분" 예요. 
# 11. "대여소"(102번 대여소가 가장 위에 와야 함)별로 "자전거번호"가 "날짜+시간"의 간격이 "30분"이상일 때 "방문코드"가 변경되는 것이 원칙이예요
# 12. "방문코드"의 시작 번호는 "A0000001"로 시작해야 하구요. "순번"은 "방문코드"에 따라서 1부터 시작해요.
# 13. "flag" 칼럼은 동일한 "방문코드"를 가지고 있을 때, 동일한 "자전거번호"가 30분 이내에 위아래 행에 위치하면 아래 행에만 "가짜"라고 적으면 돼요.
# 14. 그리고 "first data&time"과 "last data&time"은 동일한 "방문코드"의 "순번" "1"이 "first data&time"에 위치하면 되고, 
#     해당 "방문코드"의 "마지막 순번"이 "last data&time"에 위치하면 돼요. 
# 15. "flag" 칼럼의 "가짜"라는 단어때문에 코딩이 꼬여서 모든 칸이 채워지지 않았으면 좋겠어요. 해당 순번이 있는 행(row)에만 위치하면 돼요. 
# 16. "flag" 칼럼에 "가짜"라는 단어가 있어도, 동일 "방문코드"의 마지막 "순번"에서 "날짜+시간" 중 마지막이요!!
# 17. 이 파일의 이름은 "5-2_회수배송_데이터(2)"라고 지정해서 저장해 주세요.


# 아래 코드가 시간이 너무 오래 걸려, 5단계로 쪼갬
"""
import pandas as pd
from datetime import timedelta

# 8. 파일 불러오기
file_path = r"D:\pythondata\5-1_회수배송_데이터(1).csv"
df = pd.read_csv(file_path, encoding='utf-8-sig', dtype={'대여소': str, '대여소명': str})

# 날짜+시간 컬럼을 datetime 타입으로 재변환 (불러오면 object일 수 있음)
df['날짜+시간'] = pd.to_datetime(df['날짜+시간'])

# 9. 새 필드 초기화
df['방문코드'] = ''
df['순번'] = 0
df['flag'] = ''
df['자전거대수'] = 0
df['first data&time'] = ''
df['last data&time'] = ''

# 11~12. 방문코드 및 순번 생성
visit_code_counter = 1
prev_rows = {}  # key: (대여소, 자전거번호) → (last_datetime, visit_code, seq_num)

for idx, row in df.iterrows():
    key = (row['대여소'], row['자전거번호'])
    curr_time = row['날짜+시간']

    if key not in prev_rows or (curr_time - prev_rows[key][0]) > timedelta(minutes=30):
        visit_code = f"A{visit_code_counter:07d}"
        visit_code_counter += 1
        visit_seq = 1
    else:
        visit_code = prev_rows[key][1]
        visit_seq = prev_rows[key][2] + 1

    df.at[idx, '방문코드'] = visit_code
    df.at[idx, '순번'] = visit_seq
    prev_rows[key] = (curr_time, visit_code, visit_seq)

# 13. flag 처리: 동일 방문코드 내에서 같은 자전거번호가 30분 이내일 때 아래 행만 '가짜'
df = df.sort_values(by=['대여소', '자전거번호', '날짜+시간']).reset_index(drop=True)
df['flag'] = ''  # 다시 초기화

for i in range(1, len(df)):
    curr = df.iloc[i]
    prev = df.iloc[i - 1]

    if curr['자전거번호'] == prev['자전거번호'] and \
       curr['방문코드'] == prev['방문코드'] and \
       (curr['날짜+시간'] - prev['날짜+시간']) <= timedelta(minutes=30):
        df.at[i, 'flag'] = '가짜'

# 14. first / last data&time 처리
df['first data&time'] = ''
df['last data&time'] = ''

for code, group in df.groupby('방문코드'):
    group_sorted = group.sort_values(by='순번')
    first_idx = group_sorted.index[0]
    last_idx = group_sorted.index[-1]
    df.at[first_idx, 'first data&time'] = group_sorted.iloc[0]['날짜+시간']
    df.at[last_idx, 'last data&time'] = group_sorted.iloc[-1]['날짜+시간']

# 15. 자전거대수 계산 (방문코드별 고유 자전거 수)
for code, group in df.groupby('방문코드'):
    count = group['자전거번호'].nunique()
    df.loc[df['방문코드'] == code, '자전거대수'] = count

# 16. 최종 컬럼 순서 정리
final_columns = ['No', '자전거번호', '대여소', '대여소명', '날짜', '시간', '날짜+시간',
                 '방문코드', '순번', 'flag', '자전거대수',
                 'first data&time', 'last data&time', '구분']
df = df[final_columns]

# 17. 저장
output_path = r"D:\pythondata\5-2_회수배송_데이터(2).csv"
df.to_csv(output_path, index=False, encoding='utf-8-sig')

print("✅ 5-2 파일 저장 완료!", output_path)
"""

'\nimport pandas as pd\nfrom datetime import timedelta\n\n# 8. 파일 불러오기\nfile_path = r"D:\\pythondata\x05-1_회수배송_데이터(1).csv"\ndf = pd.read_csv(file_path, encoding=\'utf-8-sig\', dtype={\'대여소\': str, \'대여소명\': str})\n\n# 날짜+시간 컬럼을 datetime 타입으로 재변환 (불러오면 object일 수 있음)\ndf[\'날짜+시간\'] = pd.to_datetime(df[\'날짜+시간\'])\n\n# 9. 새 필드 초기화\ndf[\'방문코드\'] = \'\'\ndf[\'순번\'] = 0\ndf[\'flag\'] = \'\'\ndf[\'자전거대수\'] = 0\ndf[\'first data&time\'] = \'\'\ndf[\'last data&time\'] = \'\'\n\n# 11~12. 방문코드 및 순번 생성\nvisit_code_counter = 1\nprev_rows = {}  # key: (대여소, 자전거번호) → (last_datetime, visit_code, seq_num)\n\nfor idx, row in df.iterrows():\n    key = (row[\'대여소\'], row[\'자전거번호\'])\n    curr_time = row[\'날짜+시간\']\n\n    if key not in prev_rows or (curr_time - prev_rows[key][0]) > timedelta(minutes=30):\n        visit_code = f"A{visit_code_counter:07d}"\n        visit_code_counter += 1\n        visit_seq = 1\n    else:\n        visit_code = prev_rows[key][1]\n        visit_seq = prev_rows[key][2] + 1\n\n 

### ✅ 1단계: 데이터 로딩 및 기본 정리(초기화)

In [4]:
import pandas as pd

# 파일 불러오기
file_path = r"D:\pythondata\5-1_회수배송_데이터(1).csv"
df = pd.read_csv(file_path, encoding='utf-8-sig', dtype={'대여소': str, '대여소명': str})

# 날짜+시간 컬럼 재변환
df['날짜+시간'] = pd.to_datetime(df['날짜+시간'])

# 새 컬럼 초기화
df['방문코드'] = ''
df['순번'] = 0
df['flag'] = ''
df['자전거대수'] = 0
df['first data&time'] = ''
df['last data&time'] = ''

print("✅ 1단계: 데이터 로딩 및 초기화 완료 ✅")

✅ 1단계: 데이터 로딩 및 초기화 완료 ✅


### ✅ 2단계: 방문코드 및 순번 생성

In [5]:
from datetime import timedelta

df = df.sort_values(by=['대여소', '자전거번호', '날짜+시간']).reset_index(drop=True)

visit_code_list = []
visit_seq_list = []

visit_code_counter = 1
prev_key = None
prev_time = None
current_code = ''
current_seq = 0

for _, row in df.iterrows():
    key = (row['대여소'], row['자전거번호'])
    curr_time = row['날짜+시간']

    if key != prev_key or (curr_time - prev_time) > timedelta(minutes=30):
        current_code = f"A{visit_code_counter:07d}"
        visit_code_counter += 1
        current_seq = 1
    else:
        current_seq += 1

    visit_code_list.append(current_code)
    visit_seq_list.append(current_seq)

    prev_key = key
    prev_time = curr_time

df['방문코드'] = visit_code_list
df['순번'] = visit_seq_list

print("✅ 2단계: 방문코드 + 순번 생성 완료 ✅")

✅ 2단계: 방문코드 + 순번 생성 완료 ✅


### ✅ 3단계: flag 처리

In [6]:
df['flag'] = ''

for i in range(1, len(df)):
    curr = df.iloc[i]
    prev = df.iloc[i - 1]

    if curr['자전거번호'] == prev['자전거번호'] and \
       curr['방문코드'] == prev['방문코드'] and \
       (curr['날짜+시간'] - prev['날짜+시간']) <= timedelta(minutes=30):
        df.at[i, 'flag'] = '가짜'
        
print("✅ 3단계: flag 처리 완료 ✅")

✅ 3단계: flag 처리 완료 ✅


### ✅ 4단계: first / last data&time 처리

In [7]:
df['first data&time'] = ''
df['last data&time'] = ''

for code, group in df.groupby('방문코드'):
    group_sorted = group.sort_values(by='순번')
    first_idx = group_sorted.index[0]
    last_idx = group_sorted.index[-1]
    df.at[first_idx, 'first data&time'] = group_sorted.iloc[0]['날짜+시간']
    df.at[last_idx, 'last data&time'] = group_sorted.iloc[-1]['날짜+시간']
    
print("✅ 4단계: first/last data&time 처리 완료 ✅")

✅ 4단계: first/last data&time 처리 완료 ✅


### ✅ 5단계: 자전거대수 + 컬럼 순서 재정리 및 저장

In [9]:
# 자전거대수 계산
"""
for code, group in df.groupby('방문코드'):
    count = group['자전거번호'].nunique()
    df.loc[df['방문코드'] == code, '자전거대수'] = count
"""
df['자전거대수'] = df.groupby('방문코드')['자전거번호'].transform('nunique')

# 컬럼 순서 정리
final_columns = ['No', '자전거번호', '대여소', '대여소명', '날짜', '시간', '날짜+시간',
                 '방문코드', '순번', 'flag', '자전거대수',
                 'first data&time', 'last data&time', '구분']
df = df[final_columns]

# 저장
output_path = r"D:\pythondata\5-2_회수배송_데이터(2).csv"
df.to_csv(output_path, index=False, encoding='utf-8-sig')
print("✅ 5단계: 자전거대수 계산 및 저장 완료 ✅")
print("✅ 5-2 파일 저장 완료!", output_path)

✅ 5단계: 자전거대수 계산 및 저장 완료 ✅
✅ 5-2 파일 저장 완료! D:\pythondata\5-2_회수배송_데이터(2).csv
